# Install

In [ ]:
import requests
import pandas as pd
!pip install textblob
from textblob import TextBlob  # Türkçe için ayrı öneri de vereceğim
pip install streamlit
pip install googletrans==4.0.0rc1


# Main Script

In [ ]:
import requests
import pandas as pd
import os
from dotenv import load_dotenv

load_dotenv()
API_KEY = os.getenv("YOUTUBE_API_KEY")
VIDEO_ID = "pcGzWN1HEzU"

def fetch_comments(video_id, api_key):
    comments = []
    page_token = ''
    
    while True:
        url = "https://www.googleapis.com/youtube/v3/commentThreads"
        params = {
            "part": "snippet",
            "videoId": video_id,
            "key": api_key,
            "maxResults": 100,
            "textFormat": "plainText",
            "pageToken": page_token
        }

        response = requests.get(url, params=params)
        data = response.json()

        if 'items' not in data:
            print("Error:", data)
            break

        for item in data['items']:
            snippet = item['snippet']['topLevelComment']['snippet']
            comments.append(snippet['textDisplay'])

        page_token = data.get('nextPageToken')
        if not page_token:
            break

    df = pd.DataFrame(comments, columns=["comment"])
    df.to_csv("comments.csv", index=False)
    return df


# Sentiment Analysis

In [ ]:
from textblob import TextBlob
from main import get_translation  # Adjust this import if get_translation is defined elsewhere

def analyze_sentiment(text):
    translated = get_translation(text)  # Automatically translate the text to English
    blob = TextBlob(translated)
    polarity = blob.sentiment.polarity

    if polarity > 0:
        return "positive"
    elif polarity < 0:
        return "negative"
    else:
        return "neutral"


# Visualization

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud

def plot_sentiment_distribution(df):
    sns.countplot(data=df, x="sentiment")
    plt.title("Sentiment Distribution")
    plt.xlabel("Sentiment")
    plt.ylabel("Count")
    plt.show()

def generate_wordcloud(df):
    text = " ".join(df["comment"])
    wordcloud = WordCloud(width=800, height=400).generate(text)
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis("off")
    plt.title("Word Cloud")
    plt.show()


# Web Interface

In [ ]:
import streamlit as st
import pandas as pd
from main import fetch_comments
from sentiment import analyze_sentiment
from visualization import plot_sentiment_distribution, generate_wordcloud

st.title("🎥 YouTube Comment Analyzer")

video_id = st.text_input("Enter YouTube Video ID:")
if st.button("Fetch & Analyze"):
    with st.spinner("Fetching comments..."):
        df = fetch_comments(video_id, api_key="YOUR_API_KEY_HERE")
        df["sentiment"] = df["comment"].apply(analyze_sentiment)
        st.success("Comments fetched and analyzed!")

        st.subheader("📊 Sentiment Distribution")
        st.bar_chart(df["sentiment"].value_counts())

        st.subheader("☁️ Word Cloud")
        generate_wordcloud(df)

        st.dataframe(df.head())


# Dependencies File

In [ ]:
requests
pandas
textblob
seaborn
matplotlib
wordcloud
streamlit
python-dotenv


# Violin Plot for Sentiment Polarity

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

def plot_violin_sentiment(df):
    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(12, 4))

    # First plot: General distribution
    sns.violinplot(y=df["polarity"], ax=axes[0], showmeans=False, showmedians=True)
    axes[0].set_title("Overall View of Comments", fontsize=12)
    axes[0].set_ylabel("Polarity (Sentiment)", fontsize=10)
    axes[0].set_xlabel("")

    # Second plot: Same data with custom x-label
    sns.violinplot(y=df["polarity"], ax=axes[1], showmeans=False, showmedians=True)
    axes[1].set_title("Average Sentiment", fontsize=12)
    axes[1].set_ylabel("Polarity (Sentiment)", fontsize=10)
    axes[1].set_xlabel("")

    # Custom x-tick labels
    axes[0].set_xticks([0])
    axes[0].set_xticklabels(["Average"])

    axes[1].set_xticks([0])
    axes[1].set_xticklabels(["Average"])

    plt.tight_layout()
    plt.show()


In [ ]:
import matplotlib.pyplot as plt

def plot_sentiment_scatter(df):
    # Optional: Define categories if you have them (e.g. video topics or tags)
    # Otherwise use a dummy single category
    df["category"] = "YouTube Comments"  # default single category

    fig, ax = plt.subplots(figsize=(12, 6))
    
    categories = list(df["category"].unique())
    colors = ['orange', 'blue', 'black', 'yellow', 'red']
    
    for category in categories:
        sub_df = df[df["category"] == category].reset_index(drop=True)
        
        x = sub_df["polarity"]
        y = sub_df["subjectivity"]
        
        color = colors.pop() if colors else 'gray'  # fallback color
        size = 250

        ax.scatter(x, y, alpha=0.3, edgecolors='none', label=category, c=color, s=size)

    ax.set_title("Polarity vs Subjectivity of Comments")
    ax.set_xlabel("Polarity")
    ax.set_ylabel("Subjectivity")
    ax.grid(True)
    
    ax.legend(loc='upper center', bbox_to_anchor=(0.5, 1.05), ncol=3, fancybox=True, shadow=True)

    plt.tight_layout()
    plt.show()


In [ ]:
st.subheader("🔝 Top Positive Comments")
for c in df.sort_values(by="polarity", ascending=False)["comment"].head(3):
    st.markdown(f"> {c}")

st.subheader("🔻 Top Negative Comments")
for c in df.sort_values(by="polarity")["comment"].head(3):
    st.markdown(f"> {c}")
